In [8]:
import pickle
with open('dataset.pkl', 'rb') as f:
    dataset = pickle.load(f)

In [ ]:
model_name = "MODELNAME"
tkn="HF TOKEN"

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from trl import AutoModelForCausalLMWithValueHead, PPOConfig, PPOTrainer, DPOConfig, DPOTrainer

quantization_config = BitsAndBytesConfig(load_in_8bit=False)

tokenizer = AutoTokenizer.from_pretrained(model_name, token=tkn, device_map="auto")
model = AutoModelForCausalLM.from_pretrained(model_name, token=tkn,  device_map="auto")

In [ ]:
from peft import LoraConfig, PeftModel

lora_config = LoraConfig(
    r=18,
    lora_alpha=32,
    lora_dropout=0,
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
)

from trl import CPOConfig, CPOTrainer

cpo_config = CPOConfig(
    beta=0.1,
    output_dir="outputllama/",
    max_length=256,
    max_prompt_length=256,
    per_device_train_batch_size = 1,
    save_steps=500,
    loss_type="simpo",
    cpo_alpha=0
)

cpo_trainer = CPOTrainer(
    model,
    args=cpo_config,
    train_dataset=dataset,
    tokenizer=tokenizer,
    peft_config=lora_config,
)

In [ ]:
import accelerate
accelerate.notebook_launcher(cpo_trainer.train(), num_processes=4)